In [1]:
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
import os
import datetime
import psycopg2
import pandas as pd

from typing import List
from sqlalchemy import create_engine, text


In [3]:
username = os.environ["USERNAME"]
password = os.environ["PASSWORD"]
hostname = os.environ["HOSTNAME"]
database = os.environ["DATABASE"]
port = os.environ["PORT"]

db_credentials = f"postgresql://{username}:{password}@{hostname}:{port}/{database}"
engine = create_engine(db_credentials)
db_connections = engine.connect()


In [4]:
sql_query = text(
    f"""
SELECT
    provided_documents.nct_id,
    provided_documents.has_protocol AS has_protocol,
    provided_documents.has_icf AS has_icf,
    provided_documents.has_sap AS has_sap,
    provided_documents.document_date AS document_date,
    provided_documents.url AS url
FROM 
    ctgov.provided_documents
ORDER BY provided_documents.nct_id
;
"""
)

# Execute the SQL query and create a pandas DataFrame from the result
df = pd.read_sql_query(
    sql_query,
    engine,
    params={}  # type: ignore
)
df


,nct_id,has_protocol,has_icf,has_sap,document_date,url
0,NCT00000125,False,True,False,2019-02-06,https://ClinicalTrials.gov/ProvidedDocs/25/NCT...
1,NCT00001259,True,False,True,2019-06-18,https://ClinicalTrials.gov/ProvidedDocs/59/NCT...
2,NCT00001277,True,False,True,2020-12-17,https://ClinicalTrials.gov/ProvidedDocs/77/NCT...
3,NCT00001305,True,False,False,2017-04-18,https://ClinicalTrials.gov/ProvidedDocs/05/NCT...
4,NCT00001305,False,False,True,2017-04-18,https://ClinicalTrials.gov/ProvidedDocs/05/NCT...
...,...,...,...,...,...,...
40269,NCT05806606,False,True,False,2021-12-10,https://ClinicalTrials.gov/ProvidedDocs/06/NCT...
40270,NCT05806853,True,False,True,2019-10-31,https://ClinicalTrials.gov/ProvidedDocs/53/NCT...
40271,NCT05806866,True,False,True,2023-02-15,https://ClinicalTrials.gov/ProvidedDocs/66/NCT...
40272,NCT05808400,True,True,True,2023-02-14,https://ClinicalTrials.gov/ProvidedDocs/00/NCT...


In [5]:
# df.set_index(df['nct_id'], inplace=True)
# df.drop(columns=['nct_id'], inplace=True)

In [6]:
output = "ctgov_provided_documents"
timestamp = datetime.datetime.now().strftime("%Y%m%d")
file_name = f"{output}_{timestamp}.csv"
# Write the data to output filename
df.to_csv(file_name)


## Statistics

In [7]:
Total = len(df)
print(Total)

Prot = len(df[(df['has_protocol'] == True) & (df['has_icf'] == False) & (df['has_sap'] == False)])
print(Prot)
ICF = len(df[(df['has_protocol'] == False) & (df['has_icf'] == True) & (df['has_sap'] == False)])
print(ICF)
SAP = len(df[(df['has_protocol'] == False) & (df['has_icf'] == False) & (df['has_sap'] == True)])
print(SAP)

Prot_ICF = len(df[(df['has_protocol'] == True) & (df['has_icf'] == True) & (df['has_sap'] == False)])
print(Prot_ICF)
Prot_SAP = len(df[(df['has_protocol'] == True) & (df['has_icf'] == False) & (df['has_sap'] == True)])
print(Prot_SAP)

Prot_ICF_SAP = len(df[(df['has_protocol'] == True) & (df['has_icf'] == True) & (df['has_sap'] == True)])
print(Prot_ICF_SAP)


sum = Prot + ICF + SAP + Prot_ICF + Prot_SAP + Prot_ICF_SAP 
print(sum)


40274
10667
5782
8989
205
13797
834
40274


In [8]:
# # Set the maximum width for displaying column contents
# pd.set_option('display.max_colwidth', None)

# url = df['url'].to_string(index=False)
# # file object
# with open('url.txt','w') as f:
#     # write to file. 
#     f.write(url)    

In [9]:
import os
import requests
import hashlib
import wget
from tqdm.auto import tqdm
from requests.packages.urllib3.util.retry import Retry
import time

# Create 'pdf' folder if it does not exis5
if not os.path.exists('pdf'):
    os.makedirs('pdf')

offset = 690
# Assuming 'df' is your pandas DataFrame
with tqdm(total=len(df), initial=offset, desc="Processing rows") as pbar:
    for index, row in df[offset:].iterrows():
# with tqdm(total=len(df), desc="Processing rows") as pbar:
#     for index, row in df.iterrows():
        # Replace 'NCT' with the actual column name containing the document identifier
        doc_name = row['nct_id']
        has_protocol = row['has_protocol']
        has_icf = row['has_icf']
        has_sap = row['has_sap']
        # Replace 'url' with the actual column name containing the document URL
        url = row['url']

        if has_protocol:
            if has_icf and has_sap:
                doc_name += '_Prot_ICF_SAP.pdf'
            elif has_icf:
                doc_name += '_Prot_ICF.pdf'
            elif has_sap:
                doc_name += '_Prot_SAP.pdf'
            else:
                doc_name += '_Prot.pdf'
        else:
            if has_icf and has_sap:
                doc_name += '_ICF_SAP.pdf'
            elif has_icf:
                doc_name += '_ICF.pdf'
            elif has_sap:
                doc_name += '_SAP.pdf'

        file_path = os.path.join('pdf', doc_name)
        # time.sleep(20)  # Add a delay of 1 second between requests
        # # Download and save the URL as PDF
        # response = requests.get(url, allow_redirects=True)
        # open(file_path, 'wb').write(response.content)
        # wget.download(url, file_path)
        
        cmd = f"wget -q -c {url} --output-document {file_path}"
        # # cmd = f"curl --no-progress-meter -o {file_path} {url}"
        try:
            os.system(cmd)
        except Exception as e:
            print(f"An error occurred: {str(e)} of command {cmd}")
            continue
        time.sleep(10)
        pbar.update(1)
        # max_retries = 5
        # for _ in range(max_retries):
        #     try:
        #         os.system(cmd)
        #         break  # Break the loop if download is successful
        #     except Exception as e:
        #         print(f"An error occurred: {str(e)} for {file_path}")
        #         # Add additional error handling if needed

        #     # Sleep for a short time before retrying
        #     time.sleep(1)
        # pbar.update(1)  # Update progress bar


Processing rows:   0%|          | 0/40274 [00:00<?, ?it/s]

In [ ]:
# import os
# import requests
# import hashlib
# from tqdm.auto import tqdm
# from requests.packages.urllib3.util.retry import Retry
# import time

# # Create 'pdf' folder if it does not exist
# if not os.path.exists('pdf'):
#     os.makedirs('pdf')


# # Assuming 'df' is your pandas DataFrame
# with tqdm(total=len(df), desc="Processing rows") as pbar:
#     for index, row in df.iterrows():
#         # Replace 'NCT' with the actual column name containing the document identifier
#         doc_name = row['nct_id']
#         has_protocol = row['has_protocol']
#         has_icf = row['has_icf']
#         has_sap = row['has_sap']
#         # Replace 'url' with the actual column name containing the document URL
#         url = row['url']

#         if has_protocol:
#             if has_icf and has_sap:
#                 doc_name += '_Prot_ICF_SAP.pdf'
#             elif has_icf:
#                 doc_name += '_Prot_ICF.pdf'
#             elif has_sap:
#                 doc_name += '_Prot_SAP.pdf'
#             else:
#                 doc_name += '_Prot.pdf'
#         else:
#             if has_icf and has_sap:
#                 doc_name += '_ICF_SAP.pdf'
#             elif has_icf:
#                 doc_name += '_ICF.pdf'
#             elif has_sap:
#                 doc_name += '_SAP.pdf'

#         file_path = os.path.join('pdf', doc_name)
#         # Check if the file already exists and has the expected file size
#         expected_checksum = hashlib.md5(requests.get(url).content).hexdigest()
#         time.sleep(4)  # Add a delay of 1 second between requests
#         if os.path.exists(file_path):
#             with open(file_path, 'rb') as file:
#                 existing_checksum = hashlib.md5(file.read()).hexdigest()
#             if existing_checksum == expected_checksum:
#                 # File already exists and is completely downloaded. Skipping.
#                 pbar.update(1)  # Increment the progress bar
#                 continue
#             else:
#                 retries = 3  # Maximum number of retries
#                 while retries > 0:
#                     try:
#                         # Download and save the URL as PDF
#                         response = requests.get(url, stream=True)
#                         with open(file_path, 'wb') as file:
#                             for chunk in response.iter_content(chunk_size=1024):
#                                 if chunk:
#                                     file.write(chunk)
#                         break  # Break out of the retry loop if download is successful
#                     except requests.exceptions.RequestException:
#                         retries -= 1
#                         if retries == 0:
#                             print(f"Failed to download file: {doc_name}")
                
#         else:
#             retries = 3  # Maximum number of retries
#             while retries > 0:
#                 try:
#                     # Download and save the URL as PDF
#                     response = requests.get(url, stream=True)
#                     with open(file_path, 'wb') as file:
#                         for chunk in response.iter_content(chunk_size=1024):
#                             if chunk:
#                                 file.write(chunk)
#                     break  # Break out of the retry loop if download is successful
#                 except requests.exceptions.RequestException:
#                     retries -= 1
#                     if retries == 0:
#                         print(f"Failed to download file: {doc_name}")
#         pbar.update(1)


In [ ]:
# import os
# import requests
# import hashlib
# from tqdm import tqdm
# from concurrent.futures import ThreadPoolExecutor

# # Create 'pdfs' folder if it does not exist
# if not os.path.exists('pdfs'):
#     os.makedirs('pdfs')

# # Assuming 'df' is your pandas DataFrame
# file_tasks = []

# for index, row in df.iterrows():
#     # Replace 'NCT' with the actual column name containing the document identifier
#     doc_name = row['nct_id']
#     has_protocol = row['has_protocol']
#     has_icf = row['has_icf']
#     has_sap = row['has_sap']
#     # Replace 'url' with the actual column name containing the document URL
#     url = row['url']

#     if has_protocol:
#         if has_icf and has_sap:
#             doc_name += '_Prot_ICF_SAP.pdf'
#         elif has_icf:
#             doc_name += '_Prot_ICF.pdf'
#         elif has_sap:
#             doc_name += '_Prot_SAP.pdf'
#         else:
#             doc_name += '_Prot.pdf'
#     else:
#         if has_icf and has_sap:
#             doc_name += '_ICF_SAP.pdf'
#         elif has_icf:
#             doc_name += '_ICF.pdf'
#         elif has_sap:
#             doc_name += '_SAP.pdf'

#     file_path = os.path.join('pdfs', doc_name)

#     # Check if the file already exists and has the expected file size
#     # Check if the file already exists and has the same checksum
#     expected_checksum = hashlib.md5(requests.get(url).content).hexdigest()
#     if os.path.exists(file_path):
#         with open(file_path, 'rb') as file:
#             existing_checksum = hashlib.md5(file.read()).hexdigest()
#         if existing_checksum == expected_checksum:
#             print(
#                 f"File {doc_name} already exists and is completely downloaded. Skipping.")
#             continue

#     # Add file download task to the list of tasks
#     file_tasks.append((file_path, url, doc_name))

# # Download files in parallel using threads


# def download_file(file_path, url, doc_name):
#     with open(file_path, 'wb') as file:
#         response = requests.get(url, stream=True)
#         total_size = int(response.headers.get('content-length', 0))
#         with tqdm(total=total_size, unit='B', unit_scale=True, desc=f"Downloading {doc_name}") as progress_bar:
#             for chunk in response.iter_content(chunk_size=1024):
#                 if chunk:
#                     file.write(chunk)
#                     progress_bar.update(len(chunk))


# # Create a ThreadPoolExecutor with a maximum number of worker threads
# max_workers = os.cpu_count()  # Use the number of available CPU cores
# with ThreadPoolExecutor(max_workers=max_workers) as executor:
#     # Submit file download tasks to the executor
#     futures = [executor.submit(download_file, file_path, url, doc_name)
#                for file_path, url, doc_name in file_tasks]

#     # Wait for all tasks to complete
#     for future in tqdm(futures, desc="Waiting for downloads to complete"):
#         future.result()

# print("All downloads complete.")
